Instalar librerías necesarias

In [2]:
!pip install pandas
!pip install validators
!pip install newspaper3k
!pip install google
!pip install spacy
!pip install visualise_spacy_tree
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importar librerías 

In [4]:
import pandas as pd
import requests 
import re
import validators
from googlesearch import search 
from newspaper import Article

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

from spacy.matcher import Matcher 

from spacy import displacy 
import visualise_spacy_tree
from IPython.display import Image, display

# load english language model
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])

from unidecode import unidecode

Traer drive para dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Dataset de dónde tomar links

In [19]:
# Agregar dirección del archivo en pd.read_excel('/content/drive/{aqui}')
dfWithTextsToSummarize = pd.read_excel('/content/drive/')

Función para resumir textos

In [20]:
# resumir texto (texto, qué tan largo queremos que sea el resumen generado)
def summarizeText(text, per):
  try:
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary
  except:
    return "no es un articulo"

FUNCIÓN QUE TRAE TEXTO DE LINK

In [21]:
# traer contenido de un html
def bringTextFromLink(url):
  try: 
    article = Article(url)
    article.download()
    article.parse()

    return article.text
  except: 
    return "0"

FUNCIONES PARA EL PROMEDIO DE PALABRAS POR RESUMEN Y SU DESVIACIÓN ESTANDAR

In [22]:
def promPalabrasPorTexto(listaDeTextos):
    cantidadTextos = len(listaDeTextos)
    sumaTotalPalabras = 0
    for text in listaDeTextos:
      textoLimpio = text.replace("  ", " ")
      separarTextoEnListaDePalabras = textoLimpio.split(" ")
      cantidadPalabras = len(separarTextoEnListaDePalabras)
      sumaTotalPalabras += cantidadPalabras
    
    promedio = sumaTotalPalabras/cantidadTextos
    return promedio

def desvEstandar(prom, listaDeTextos):
  cantidadTextos = len(listaDeTextos)
  sumatoriaNumerador = 0
  for text in listaDeTextos:
    textoLimpio = text.replace("  ", " ")
    separarTextoEnListaDePalabras = textoLimpio.split(" ")
    cantidadPalabras = len(separarTextoEnListaDePalabras)
    restaYCuadrado = (cantidadPalabras-prom)**2
    sumatoriaNumerador += restaYCuadrado
  
  desvEst = pow((sumatoriaNumerador/cantidadTextos), (1/2))
  return desvEst


CASO 1: CUANDO SOLO SE TIENEN LAS REFERENCIAS

In [ ]:
# Crear columna en dataset para agregar los resumenes 
nombreDeLaColumnaParaLosResumenes = ""
dfWithTextsToSummarize[nombreDeLaColumnaParaLosResumenes] = ""

# Agregar en variable columna Referencias la columna en dónde están almacenadas la referencia o las referencias
columnaReferencias = ""
# Agregar el caracter que separa las referencias
caracterQueSeparaLasReferencias = ""

# Porcentaje al que se desea reducir el texto Ej. .20 -> Rango del 0 al 1
porcenatajeAReducirTextos = .20

# El siguiente for itera sobre cada caso del dataset
for index, row in dfWithTextsToSummarize.iterrows():
  # Cambiar nombre de la columna en dónde esta la referencia o están las referencias 
  referencias = str(row[columnaReferencias])
  # la siguiente linea separará las referencias según el caracter que se haya usado para separar los links
  listaReferencias = referencias.split(caracterQueSeparaLasReferencias)
  
  listaResumenesPorCaso = []
  # El siguiente for itera sobre cada referencia para obtener su texto 
  for ref in listaReferencias:
    textoLimpioDeLaReferencia = bringTextFromLink(ref)
    resumen = summarizeText(textoLimpioDeLaReferencia, porcenatajeAReducirTextos)
    if resumen != "0":
      listaResumenesPorCaso.append(resumen)
  
  # La siguiente linea concatena la lista de resumenes con el caracter que se introduzca en la variable "caracterSeparaResumenes"
  caracterSeparaResumenes = ""
  resumenesConcatenados = str(f" {caracterSeparaResumenes} ").join(listaResumenesPorCaso)

  # Agregar resumen concatenado al caso en la columna correspondiente
  dfWithTextsToSummarize.at[index, nombreDeLaColumnaParaLosResumenes] = resumenesConcatenados

Building prefix dict from /usr/local/lib/python3.9/dist-packages/jieba/dict.txt ...
DEBUG:jieba:Building prefix dict from /usr/local/lib/python3.9/dist-packages/jieba/dict.txt ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.4495165348052979 seconds.
DEBUG:jieba:Loading model cost 1.4495165348052979 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


CASO 2: CUANDO YA SE TIENE EL TEXTO POR CASO QUE SE DESA RESUMIR

In [ ]:
# Crear columna en dataset para agregar los resumenes 
nombreDeLaColumnaParaLosResumenes = ""
dfWithTextsToSummarize[nombreDeLaColumnaParaLosResumenes] = ""

# Definir nombre de la columna en dónde se encuentran los textos que se desean resumir
nombreColumnaTexto = ""

# Porcentaje al que se desea reducir el texto Ej. .20 -> Rango del 0 al 1
porcenatajeAReducirTextos = .20

# El siguiente for itera sobre cada caso del dataset
for index, row in dfWithTextsToSummarize.iterrows():
  texto = str(row[nombreColumnaTexto])
  resumen = summarizeText(texto, porcenatajeAReducirTextos)
    
  # Agregar resumen al caso en la columna correspondiente
  dfWithTextsToSummarize.at[index, nombreDeLaColumnaParaLosResumenes] = resumen

In [ ]:
dfWithTextsToSummarize.head()

VISUALIZAR DATASET UNA VEZ CON RESUMENES

DATOS IMPORTANTES SOBRE LOS RESUMENES

In [ ]:
columnaConResumenes = list(dfWithTextsToSummarize[nombreDeLaColumnaParaLosResumenes])
promedio = promPalabrasPorTexto(columnaConResumenes)
desviacionEstandar = desvEstandar(promedio, columnaConResumenes)

print(f"Promedio de palabras por resumen: {promedio}")
print(f"Desviación estándar: {desviacionEstandar}")

Exportar nuevo dataset con resumenes

In [ ]:
# En la siguiente variable asignar el nombre que se quiere que tenga el archivo
nombreArchivo = "EjemploConTextosResumidos"
dfWithTextsToSummarize.to_excel(f'{nombreArchivo}.xlsx')